In [18]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import sklearn.metrics as skm
from sklearn.metrics import accuracy_score, confusion_matrix
rs = {'random_state': 42}

import test_coink.utils.paths as path
pd.options.display.float_format = '{:,.2f}'.format

In [19]:
# Cargar los datos
data = pd.read_csv(path.data_processed_dir('info_satisfaccion_trabajo_clean.csv'))
data.sample(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
995,43,False,Travel_Rarely,930,Research & Development,6,Bachelor,Medical,1402,Low,...,Excellent,Low,0,20,3,Bad,20,7,1,8
721,50,False,Travel_Rarely,939,Research & Development,24,Bachelor,Life Sciences,1005,Very High,...,Excellent,Very High,1,22,2,Better,12,11,1,5
244,45,False,Travel_Rarely,252,Research & Development,1,Bachelor,Other,336,High,...,Excellent,High,1,25,2,Better,24,0,1,7
75,31,False,Travel_Rarely,746,Research & Development,8,Master,Life Sciences,98,High,...,Outstanding,Very High,0,11,2,Better,11,7,1,8
395,43,False,Travel_Frequently,313,Research & Development,21,Bachelor,Medical,525,Very High,...,Outstanding,Low,1,8,1,Better,3,2,1,2


In [20]:
# fiajmos un orde estrictopar el paso de la prediccion
columns_order = ['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']
data = data[columns_order]

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   bool  
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   object
 7   EducationField            1470 non-null   object
 8   EmployeeNumber            1470 non-null   int64 
 9   EnvironmentSatisfaction   1470 non-null   object
 10  Gender                    1470 non-null   object
 11  HourlyRate                1470 non-null   int64 
 12  JobInvolvement            1470 non-null   object
 13  JobLevel                  1470 non-null   int64 
 14  JobRole                 

In [22]:
# Crear un objeto ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'DailyRate', 'DistanceFromHome', 'EmployeeNumber', 'HourlyRate','JobLevel', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
                                    'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears','TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole',
                                    'YearsSinceLastPromotion', 'YearsWithCurrManager']),

        ('num_pca', PCA(n_components=3), ['JobLevel','MonthlyIncome','YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion','YearsWithCurrManager']),

        ('cat', OneHotEncoder(), ['BusinessTravel', 'Department', 'Education','EducationField', 'EnvironmentSatisfaction', 'Gender', 'JobInvolvement',
        'JobRole', 'MaritalStatus', 'Over18', 'OverTime','PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance'])
    ],
    remainder= 'passthrough'
    )

In [23]:
# Crear el pipeline integrado
pipe = Pipeline([
    ('preprocessor', preprocessor)
])

# Dividir los datos en variables de entrada y salida
X = data.drop('JobSatisfaction', axis=1)
y = data['JobSatisfaction']

X_transform = pipe.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el pipeline integrado
# pipe.fit_transform(X_train, y_train)

clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)


100%|██████████| 29/29 [00:05<00:00,  5.01it/s]


In [24]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreeClassifier,0.29,0.28,None,0.29,0.02
LabelSpreading,0.27,0.27,None,0.28,0.18
LabelPropagation,0.27,0.27,None,0.28,0.12
Perceptron,0.24,0.26,None,0.24,0.03
RandomForestClassifier,0.30,0.26,None,0.27,0.34
CalibratedClassifierCV,0.33,0.25,None,0.24,1.47
XGBClassifier,0.27,0.25,None,0.26,0.65
DummyClassifier,0.33,0.25,None,0.16,0.03
LinearDiscriminantAnalysis,0.28,0.25,None,0.27,0.08


In [25]:
# Classification - Model Pipeline
def modelPipeline(X_train, X_test, y_train, y_test):

    log_reg = LogisticRegression(**rs)
    nb = BernoulliNB()
    knn = KNeighborsClassifier()
    svm = SVC(**rs)
    mlp = MLPClassifier(max_iter=500, **rs)
    dt = DecisionTreeClassifier(**rs)
    et = ExtraTreesClassifier(**rs)
    rf = RandomForestClassifier(**rs)
    xgb = XGBClassifier(**rs, verbosity=0)

    # Crear un objeto ColumnTransformer
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'DailyRate', 'DistanceFromHome', 'EmployeeNumber', 'HourlyRate','JobLevel', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
                                    'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears','TrainingTimesLastYear', 'YearsAtCompany', 'YearsInCurrentRole',
                                    'YearsSinceLastPromotion', 'YearsWithCurrManager']),

        ('num_pca', PCA(n_components=3), ['JobLevel','MonthlyIncome','YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion','YearsWithCurrManager']),

        ('cat', OneHotEncoder(), ['BusinessTravel', 'Department', 'Education','EducationField', 'EnvironmentSatisfaction', 'Gender', 'JobInvolvement',
        'JobRole', 'MaritalStatus', 'Over18', 'OverTime','PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance'])
    ],
    remainder= 'passthrough'
    )

    clfs = [
            ('Logistic Regression', log_reg), 
            ('Naive Bayes', nb),
            ('K-Nearest Neighbors', knn), 
            ('SVM', svm), 
            ('MLP', mlp), 
            ('Decision Tree', dt), 
            ('Extra Trees', et), 
            ('Random Forest', rf), 
            ('XGBoost', xgb)
            ]


    pipelines = []

    scores_df = pd.DataFrame(columns=['Model', 'F1_Score', 'Precision', 'Recall', 'Accuracy', 'ROC_AUC'])


    for clf_name, clf in clfs:

        pipeline = Pipeline(steps=[
                                   ('preprocessor', preprocessor),
                                   ('classifier', clf)
                                   ]
                            )
        pipeline.fit(X_train, y_train)


        y_pred = pipeline.predict(X_test)
        # F1-Score
        fscore = skm.f1_score(y_test, y_pred,average='weighted')
        # Precision
        pres = skm.precision_score(y_test, y_pred,average='weighted')
        # Recall
        rcall = skm.recall_score(y_test, y_pred,average='weighted')
        # Accuracy
        accu = skm.accuracy_score(y_test, y_pred)
        # ROC_AUC
        # roc_auc = skm.roc_auc_score(y_test, y_pred,multi_class='ovr' )#average='weighted'


        pipelines.append(pipeline)

        scores_df = scores_df.append({
                                      'Model' : clf_name, 
                                      'F1_Score' : fscore,
                                      'Precision' : pres,
                                      'Recall' : rcall,
                                      'Accuracy' : accu
                                    #   'ROC_AUC' : roc_auc
                                      }, 
                                     ignore_index=True)
        
    return pipelines, scores_df

In [26]:
pipelines, scores_df = modelPipeline(X_train, X_test, y_train, y_test)

In [27]:
scores_df

,Model,F1_Score,Precision,Recall,Accuracy,ROC_AUC
0,Logistic Regression,0.25,0.25,0.28,0.28,NaN
1,Naive Bayes,0.26,0.27,0.27,0.27,NaN
2,K-Nearest Neighbors,0.24,0.24,0.24,0.24,NaN
3,SVM,0.18,0.18,0.31,0.31,NaN
4,MLP,0.28,0.32,0.34,0.34,NaN
5,Decision Tree,0.29,0.30,0.29,0.29,NaN
6,Extra Trees,0.26,0.25,0.28,0.28,NaN
7,Random Forest,0.24,0.23,0.27,0.27,NaN
8,XGBoost,0.24,0.23,0.24,0.24,NaN


Esta tabla muestra los resultados de 9 modelos diferentes de ML en un problema de clasificación. Las métricas incluyen F1 Score, Precision, Recall y Accuracy. Estas métricas permiten evaluar la capacidad de los modelos para clasificar correctamente las etiquetas de la clase objetivo. 
F1 Score es una métrica combinada de Precision y Recall. 
Precision mide la cantidad de etiquetas positivas correctas en comparación con las etiquetas positivas totales predichas. 
Recall mide la cantidad de etiquetas positivas correctas en comparación con las etiquetas positivas totales reales. 
Accuracy mide la proporción de etiquetas clasificadas correctamente. 

En general, la tabla muestra que ninguno de los modelos tuvo un desempeño sobresaliente en todas las métricas, lo que sugiere que podría ser necesario realizar más investigaciones y ajustes en los modelos.

### En menos 150 palabras explique cómo monitorearía este modelo en caso en caso de que estuviera desplegado para uso recurrente de la empresa.
Plataformas de monitoreo en tiempo real, como Datadog o New Relic, son muy útiles para monitorear el rendimiento de modelos de ML. Estas plataformas permiten visualizar métricas en tiempo real y detectar problemas de forma temprana. Además, permiten recibir alertas en caso de que se produzcan desviaciones importantes en el rendimiento. También proporcionan informes detallados y análisis para entender la causa de los problemas. Es importante elegir la plataforma adecuada en función de los requisitos específicos del proyecto y de las necesidades de la empresa. Con una buena plataforma de monitoreo, la empresa puede estar segura de que su modelo de clasificación está funcionando de manera óptima y de que puede detectar problemas de forma temprana.

## otra aproximacion seria

Si tu variable objetivo (y) es categórica, en lugar de utilizar un ANOVA para hacer una comparación de medias de las variables independientes, se puede utilizar un test estadístico para comparar la distribución de las variables independientes para cada categoría de la variable objetivo. Una opción popular es el test chi-cuadrado, que se puede calcular utilizando la función scipy.stats.chi2_contingency. Este test permite determinar si existe una asociación significativa entre la variable objetivo y cada una de las variables independientes.

In [46]:
from scipy.stats import chi2_contingency

p_value = []
cols = data.drop('JobSatisfaction', axis=1).columns
for col in cols:
    # Crea las tablas de contingencia para cada variable independiente
    contingency_table_1 = pd.crosstab(data['JobSatisfaction'], data[col])

    # Realiza la prueba de chi-cuadrado para cada tabla de contingencia
    chi2, p_value_1, dof, expected_values_1 = chi2_contingency(contingency_table_1)
    if p_value_1< 0.1:
        p_value.append({col:p_value_1})
        # Imprime los resultados
        print(f'P-value variable {col}: {p_value_1}')

P-value variable Attrition: 0.0005563004510387556


Como resultados observamos que solo la variable Attrition es eimportante para poder predecir JobSatisfaction y las demas no aportan mucha informacion.